In [1]:
import json
from pathlib import Path
import numpy as np
from pack import unpack_tensortrain
from multilevel2 import MultilevelSurrogate
from darcy1d import Darcy1D
from tqdm import tqdm

In [2]:
def compute_surrogates(L, main_dir, args):
    base_dir = main_dir / "base_tensors"
    refine_dir = main_dir / "refinement_tensors"

    num_samplesizes = len(args["samplesizes"])

    surrogates = []
    work_estimates = []
    for start_idx in range(L - 1, num_samplesizes):
        u = MultilevelSurrogate(d=args["d"], L=L)

        N = args["samplesizes"][start_idx]
        l = args["max_discr_level"] - L + 1

        with np.load(base_dir / f"N{N}.npz") as data:
            # print(data.files)
            ttcomponents_packed = data[f"l{l}"]
        ttcomponents = unpack_tensortrain(ttcomponents_packed)

        u.set_layer(0, ttcomponents)
        work = N * 2**l

        for layer_idx in range(1, L):
            N = args["samplesizes"][start_idx - layer_idx]
            l += 1
            work += N + (2**l + 2 ** (l - 1))

            with np.load(refine_dir / f"N{N}.npz") as data:
                # print(data.files)
                ttcomponents_packed = data[f"l{l}"]
            ttcomponents = unpack_tensortrain(ttcomponents_packed)
            u.set_layer(layer_idx, ttcomponents)

        surrogates.append(u)
        work_estimates.append(work)
    return surrogates, work_estimates

In [ ]:
testsize = 1_000
test_discr_level = 14
test_discr_degree = 2
testpoints = np.random.uniform(-1, 1, (testsize, d)
testvalues = np.zeros(testsize)
testvalues_coarse = np.zeros(testsize)
testproblem = Darcy1D(test_discr_level, args["d"], test_discr_degree)
testproblem_coarse = Darcy1D(10, args["d"], 1)

for i in tqdm(range(testsize)):
    testvalues[i] = testproblem.get_integrated_solution(testpoints[i])
    testvalues_coarse[i] = testproblem_coarse.get_integrated_solution(testpoints[i])

NameError: name 'args' is not defined

In [ ]:
import matplotlib.pyplot as plt


def plot_experiment(main_dir, L_max, testpoints, testvalues, testvalues_coarse):
    main_dir = Path(main_dir)
    with open(main_dir / "experiment_args.json", "r") as f:
        args = dict(json.load(f))

    print(args)

    plt.figure()
    als_name = args["als"].upper()
    max_discr_level = args["max_discr_level"]
    d = args["d"]
    weights = r"$w_{\nu} = \prod_j\sqrt{2\nu_j + 1}$"
    if args["radius_option"] == 2:
        weights = r"$w_{\nu} = \prod_j \sqrt{2\nu_j + 1}\, 1.01\, \nu_j^{3/4}$"
    plt.title(f"{als_name}, dim={d}, finest mesh size: 2^{max_discr_level}, {weights}")

    fem_error = np.sqrt(np.mean((testvalues - testvalues_coarse) ** 2))
    plt.axhline(y=fem_error, color="gray", linestyle="--", label="FEM error")

    for L in range(1, L_max + 1):
        surrogates, work_estimates = compute_surrogates(
            L, main_dir, args
        )  # work_estimates: shape (k,)
        work_estimates = np.asarray(work_estimates)

        preds = np.array([u(testpoints) for u in surrogates])  # shape (k, n_test)
        rmse = np.sqrt(
            np.mean((preds - testvalues[None, :]) ** 2, axis=1)
        )  # shape (k,)

        # optional: sort by work so the lines look nice
        order = np.argsort(work_estimates)
        w_sorted = work_estimates[order]
        rmse_sorted = rmse[order]

        if L == 1:
            als_rmse = np.sqrt(
                np.mean((preds - testvalues_coarse[None, :]) ** 2, axis=1)
            )
            als_rmse_sorted = als_rmse[order]
            plt.loglog(w_sorted, als_rmse_sorted, "--", label=f"{als_name} error")

        plt.loglog(w_sorted, rmse_sorted, "o-", label=f"L = {L}")

    plt.xlabel("work")
    plt.ylabel("RMSE")
    plt.grid(True, which="both", ls=":")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_experiment("experiments0/SALS1", 5, testpoints, testvalues, testvalues_coarse)
plot_experiment("experiments0/SALS2", 5, testpoints, testvalues, testvalues_coarse)
plot_experiment("experiments0/SSALS1", 5, testpoints, testvalues, testvalues_coarse)
plot_experiment("experiments0/SSALS2", 5, testpoints, testvalues, testvalues_coarse)